In [2]:
# import SimpleITK as sitk
# import jax.numpy as jnp
# import itertools


# def join_ct_suv(ct: sitk.Image, suv: sitk.Image,ct1: sitk.Image, suv1: sitk.Image) -> sitk.Image:
#     '
#     Resample a CT image to the same size as a SUV image
#     '
   
#     ct_arr=sitk.GetArrayFromImage(ct)
#     suv_arr=sitk.GetArrayFromImage(suv)

#     ct_arr_1=sitk.GetArrayFromImage(ct1)
#     suv_arr_1=sitk.GetArrayFromImage(suv1)
    
#     res=jnp.stack([jnp.array(suv_arr),jnp.array(ct_arr),jnp.array(ct_arr_1),jnp.array(suv_arr_1)],axis=-1)
#     return res

# def load_landmark_data(folder_path:str):
#     '
#     given path to folder with landmarks files and images after general registaration we load the data
#     we want to first load the suv and ct images resample them to the same size and then load the landmarks
#     we need to load separately study 0 and 1 
#     the output should be in form of a dictionary with keys 'study_0','study_1','From`,`To`' where `From` and `To` are the landmarks
#     all the data should be in form of jnp.arrays
#     '
#     ct_0=sitk.ReadImage(folder_path+'/study_0_ct_soft.nii.gz')
#     suv_0=sitk.ReadImage(folder_path+'/study_0_SUVS.nii.gz')
#     # Resample ct_0 to match ct_1
            
#     ct_1=sitk.ReadImage(folder_path+'/study_1_ct_soft.nii.gz')
#     suv_1=sitk.ReadImage(folder_path+'/study_1_SUVS.nii.gz')    
#     arr_0 = join_ct_suv(ct_0, suv_0,ct_1, suv_1)

#     return {'study':arr_0, 'From':jnp.load(folder_path+'/From.npy'),'To':jnp.load(folder_path+'/To.npy')}

In [3]:
# import os
# import numpy as np
# import numpy as np
# folder_path='/root/data/prepared_registered'
# # folder_path='/root/data/prepared_registered/pat_2/general_transform'
# # load_landmark_data(folder_path)





# def reshape_image(arr, img_size):
#     # Get the current shape of the input array
#     img_size=(img_size[1],img_size[2],img_size[3],img_size[4])
#     current_shape = arr.shape
    
#     # Check if the current shape is already equal to the desired shape
#     if current_shape == img_size:
#         print("The input array already has the desired shape.")
#         return arr
    
#     # Check if the current shape is larger than the desired shape in any dimension
#     if any(cs > ds for cs, ds in zip(current_shape, img_size)):
#         # Crop the input array from the end of the dimension where it occurs
#         arr = arr[:img_size[0], :img_size[1], :img_size[2], :img_size[3]]
#         print("The input array has been cropped to the desired shape.")
    
#     # Check if the current shape is smaller than the desired shape in any dimension
#     if any(cs < ds for cs, ds in zip(current_shape, img_size)):
#         # Pad the input array with zeros at the end of the dimension where it occurs

#         arr = np.pad(arr, ((0, np.max(img_size[0] - current_shape[0],0)),
#                                   (0, np.max(img_size[1] - current_shape[1],0)),
#                                   (0, np.max(img_size[2] - current_shape[2],0)),
#                                   (0, 0)), mode='constant')
#         print("The input array has been padded to the desired shape.")
    
#     # If none of the above conditions are met, return the input array as is
#     return arr

# batch_size=2
# img_size = (batch_size,488, 200, 200,2)

# def stack_with_pad(arr_0,arr_1):
#     if arr_0.shape[0] > arr_1.shape[0]:
#         pad_length = arr_0.shape[0] - arr_1.shape[0]
#         padding = jnp.full((pad_length, arr_1.shape[1]), -1)
#         arr_1 = jnp.concatenate((arr_1, padding), axis=0)
#     elif arr_1.shape[0] > arr_0.shape[0]:
#         pad_length = arr_1.shape[0] - arr_0.shape[0]
#         padding = jnp.full((pad_length, arr_0.shape[1]), -1)
#         arr_0 = jnp.concatenate((arr_0, padding), axis=0)
    
#     return jnp.stack([arr_0, arr_1])

    


# def get_batched(folder_tuple):
#     folder_0=load_landmark_data(f"{folder_tuple[0]}/general_transform")
#     folder_1=load_landmark_data(f"{folder_tuple[1]}/general_transform")
#     arr=jnp.stack([reshape_image(folder_0['study'],img_size),reshape_image(folder_1['study'],img_size)])
#     From=stack_with_pad(folder_0['From'],folder_1['From'])
#     To=stack_with_pad(folder_0['To'],folder_1['To'])
#     return {'study':arr, 'From':From,'To':To}


# # folder_tuples = list(itertools.zip_longest(*[iter(folder_names)] * 2))
# # tt=list(map(get_batched,folder_tuples))

# # tt=list(map(lambda el: reshape_image(load_landmark_data(f"{el}/general_transform")['study'],img_size) ,folder_names))

# # create a function that given input array'arr' will change it shape to shape given as 'img_size' if the given image is bigger than 'img_size' in any dimension image should be cropped from the end of dimension where it happend in case when image is bigger than 'img_size' image should be padded with zeros at the end of dimension where it happens ; check weather the resulting image has shape required


In [2]:
import pandas as pd
import numpy as np

radiomics_full_data_path="/workspaces/pilot_lymphoma/data/extracted_features_pet_trimmedB.csv"
radiomics_full_data=pd.read_csv(radiomics_full_data_path)
radiomics_full_data = radiomics_full_data.loc[:, ~radiomics_full_data.columns.str.contains('Unnamed', case=False)]
radiomics_full_data = radiomics_full_data[radiomics_full_data['lesion_num'] == 1000]
radiomics_full_data["pat_id"]=radiomics_full_data["pat_id"].astype(int)
radiomics_full_data.columns

Index(['pat_id', 'lesion_num', 'study_0_or_1', 'Deauville', 'lab_path',
       'mod_name', 'vol_in_mm3', 'original_firstorder_10Percentile_pet',
       'original_firstorder_90Percentile_pet',
       'original_firstorder_Energy_pet',
       ...
       'wavelet-LLL_glszm_LargeAreaLowGrayLevelEmphasis_ct',
       'wavelet-LLL_glszm_LowGrayLevelZoneEmphasis_ct',
       'wavelet-LLL_glszm_SizeZoneNonUniformity_ct',
       'wavelet-LLL_glszm_SizeZoneNonUniformityNormalized_ct',
       'wavelet-LLL_glszm_SmallAreaEmphasis_ct',
       'wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis_ct',
       'wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis_ct',
       'wavelet-LLL_glszm_ZoneEntropy_ct',
       'wavelet-LLL_glszm_ZonePercentage_ct',
       'wavelet-LLL_glszm_ZoneVariance_ct'],
      dtype='object', length=1879)

In [3]:
full_data_table_path="/workspaces/pilot_lymphoma/data/full_table_data_for_delta.csv"
full_data_table= pd.read_csv(full_data_table_path)
full_data_table["pat_id"]=full_data_table["Unnamed: 0"].astype(int)
full_data_table["outcome"]=full_data_table["Unnamed: 12"]
np.unique(full_data_table["outcome"].to_numpy)


array([<bound method IndexOpsMixin.to_numpy of 0     PR
       1     CR
       2     PR
       3     CR
       4     PD
       5     CR
       6     CR
       7     SD
       8     CR
       9     CR
       10    PD
       11    PD
       12    PR
       13    PD
       14    PR
       15    SD
       16    CR
       17    CR
       18    CR
       19    CR
       20    CR
       21    CR
       22    SD
       23    PD
       24    CR
       25    PD
       26    CR
       27    CR
       28    PD
       29    CR
       30    CR
       31    PR
       Name: outcome, dtype: object>                   ], dtype=object)

In [14]:

def subtract_dicts(dict1, dict2,dict_sums):
    # Create a new dictionary with the absolute difference of each entry
    result = {key: abs(dict1[key] - dict2[key])/dict_sums[key] for key in dict1}
    return result



def get_delta_radiomics(full_data_table_row, radiomics_full_data):

    """a function 'get_delta_radiomics' that would have two arguments 'full_data_table_row' and 'radiomics_full_data'.  'full_data_table_row' is a row from main table and contains columns like '[pat_id,outcome]'   'radiomics_full_data' contains multiple column including'[pat_id,study_0_or_1]'  Function should perform all steps:
    1) find 2 rows from 'radiomics_full_data' where  value of column 'pat_id' would be the same as value of column 'pat_id' in 'full_data_table_row' 
    2) From those 2 rows you found drop columns with names: ```['pat_id', 'lesion_num', 'study_0_or_1', 'Deauville', 'lab_path', 'mod_name']```
    3) Save the sum of both rows so each column should have sum of 2 rows
    4) calculate the absolute value of the diffrence between two rows and divide it by the saved sum save information as dictionary called 'res'
    5) add outcome variable to 'res' that you will find in column 'outcome' in 'full_data_table_row' encode the  'outcome' as integer as in the dictionary {'CR':0, 'PD':1, 'PR':2, 'SD':2, }
    6) return calculated dictionary res"""
    full_data_table_row=full_data_table_row[1]
    # print(f"pppp {full_data_table_row['pat_id']}")
    # Step 1
    rows = radiomics_full_data[radiomics_full_data['pat_id'] == full_data_table_row['pat_id']]
    if(len(rows))<2:
        print(f"ggggggggggggg {len(rows)}  {full_data_table_row['pat_id']}")
        return " "
    # Step 2
    rows = rows.drop(columns=['pat_id', 'lesion_num', 'study_0_or_1', 'Deauville', 'lab_path', 'mod_name'])
    # Step 3
    row_sum = rows.sum().to_dict()
   
    # print(len(rows))
    # Step 4
    res = subtract_dicts(rows.iloc[0].to_dict(),rows.iloc[1].to_dict(),row_sum )
    # print(res)
    # Step 5
    outcome_dict = {'CR':0, 'PD':1, 'PR':2, 'SD':2}
    res['outcome'] = outcome_dict[full_data_table_row['outcome']]

    # Step 6
    return res

rows = list(full_data_table.iterrows())


delta_r=list(map( lambda el: get_delta_radiomics(el, radiomics_full_data),rows))
delta_r= [el for el in delta_r if el != " "]
delta_r=pd.DataFrame(delta_r)



ggggggggggggg 0  1
ggggggggggggg 1  8
ggggggggggggg 1  10
ggggggggggggg 1  14
ggggggggggggg 1  15
ggggggggggggg 1  16
ggggggggggggg 0  17
ggggggggggggg 1  18
ggggggggggggg 1  21
ggggggggggggg 0  26
ggggggggggggg 0  27
ggggggggggggg 0  32
ggggggggggggg 0  33


Index(['vol_in_mm3', 'original_firstorder_10Percentile_pet',
       'original_firstorder_90Percentile_pet',
       'original_firstorder_Energy_pet', 'original_firstorder_Entropy_pet',
       'original_firstorder_InterquartileRange_pet',
       'original_firstorder_Kurtosis_pet', 'original_firstorder_Maximum_pet',
       'original_firstorder_MeanAbsoluteDeviation_pet',
       'original_firstorder_Mean_pet',
       ...
       'wavelet-LLL_glszm_LowGrayLevelZoneEmphasis_ct',
       'wavelet-LLL_glszm_SizeZoneNonUniformity_ct',
       'wavelet-LLL_glszm_SizeZoneNonUniformityNormalized_ct',
       'wavelet-LLL_glszm_SmallAreaEmphasis_ct',
       'wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis_ct',
       'wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis_ct',
       'wavelet-LLL_glszm_ZoneEntropy_ct',
       'wavelet-LLL_glszm_ZonePercentage_ct',
       'wavelet-LLL_glszm_ZoneVariance_ct', 'outcome'],
      dtype='object', length=1874)

In [18]:
# np.unique(delta_r['outcome'])

delta_r['outcome']

0     0
1     2
2     0
3     1
4     0
5     0
6     0
7     1
8     1
9     2
10    0
11    0
12    0
13    2
14    1
15    0
16    0
17    1
18    0
Name: outcome, dtype: int64

# Check what is the sum of euclidean distances for diffrent registrations


In [4]:
folder_names = [os.path.join(folder_path, name) for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))]
folder_names= list(filter(lambda el: "pat" in el, folder_names))
folder_names

['/root/data/prepared_registered/pat_12',
 '/root/data/prepared_registered/pat_16',
 '/root/data/prepared_registered/pat_31',
 '/root/data/prepared_registered/pat_21',
 '/root/data/prepared_registered/pat_26',
 '/root/data/prepared_registered/pat_2',
 '/root/data/prepared_registered/pat_19',
 '/root/data/prepared_registered/pat_13',
 '/root/data/prepared_registered/pat_10',
 '/root/data/prepared_registered/pat_28',
 '/root/data/prepared_registered/pat_8',
 '/root/data/prepared_registered/pat_24',
 '/root/data/prepared_registered/pat_15',
 '/root/data/prepared_registered/pat_4',
 '/root/data/prepared_registered/pat_29',
 '/root/data/prepared_registered/pat_14',
 '/root/data/prepared_registered/pat_20',
 '/root/data/prepared_registered/pat_22',
 '/root/data/prepared_registered/pat_5',
 '/root/data/prepared_registered/pat_18',
 '/root/data/prepared_registered/pat_11',
 '/root/data/prepared_registered/pat_9',
 '/root/data/prepared_registered/pat_27',
 '/root/data/prepared_registered/pat_3'

In [7]:
### checking from linear folder the distance
def get_dist_0(fold_name):
  fold=f"{fold_name}/lin_transf"
  fromm=np.load(f"{fold}/From.npy")
  too=np.load(f"{fold}/To.npy")

  res=(fromm-too)
  res=res*(fromm>0)
  res=np.sqrt(np.sum(res**2,axis=-1))
  return np.sum(res.flatten())

np.mean(list(map(get_dist_0,folder_names)))

27.419910917593374

In [2]:
import SimpleITK as sitk
import six
from radiomics import featureextractor, getTestCase
import multiprocessing
import optuna
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from ngboost import NGBRegressor
from sklearn.metrics import mean_squared_error
from ngboost.distns import Exponential, Normal
from ngboost import NGBClassifier
from ngboost.distns import k_categorical, Bernoulli
from mrmr import mrmr_classif
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import sklearn
import pickle
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold


def subtract_dicts(dict1, dict2,dict_sums):
    # Create a new dictionary with the absolute difference of each entry
    result = {key: abs(dict1[key] - dict2[key])/dict_sums[key] for key in dict1}
    return result



def get_delta_radiomics(full_data_table_row, radiomics_full_data):

    """a function 'get_delta_radiomics' that would have two arguments 'full_data_table_row' and 'radiomics_full_data'.  'full_data_table_row' is a row from main table and contains columns like '[pat_id,outcome]'   'radiomics_full_data' contains multiple column including'[pat_id,study_0_or_1]'  Function should perform all steps:
    1) find 2 rows from 'radiomics_full_data' where  value of column 'pat_id' would be the same as value of column 'pat_id' in 'full_data_table_row' 
    2) From those 2 rows you found drop columns with names: ```['pat_id', 'lesion_num', 'study_0_or_1', 'Deauville', 'lab_path', 'mod_name']```
    3) Save the sum of both rows so each column should have sum of 2 rows
    4) calculate the absolute value of the diffrence between two rows and divide it by the saved sum save information as dictionary called 'res'
    5) add outcome variable to 'res' that you will find in column 'outcome' in 'full_data_table_row' encode the  'outcome' as integer as in the dictionary {'CR':0, 'PD':1, 'PR':2, 'SD':2, }
    6) return calculated dictionary res"""
    full_data_table_row=full_data_table_row[1]
    # print(f"pppp {full_data_table_row['pat_id']}")
    # Step 1
    rows = radiomics_full_data[radiomics_full_data['pat_id'] == full_data_table_row['pat_id']]
    if(len(rows))<2:
        # print(f"ggggggggggggg {len(rows)}  {full_data_table_row['pat_id']}")
        return " "
    # Step 2
    rows = rows.drop(columns=['pat_id', 'lesion_num', 'study_0_or_1', 'Deauville', 'lab_path', 'mod_name'])
    # Step 3
    row_sum = rows.sum().to_dict()
   
    # print(len(rows))
    # Step 4
    res = subtract_dicts(rows.iloc[0].to_dict(),rows.iloc[1].to_dict(),row_sum )
    # print(res)
    # Step 5
    outcome_dict = {'CR':0, 'PD':1, 'PR':0, 'SD':0}
    res['outcome'] = outcome_dict[full_data_table_row['outcome']]

    # Step 6
    return res





# mrmr_selection,shap,ngboost


def display_probs(curr_class, inferred_probs, Y_test,to_be_sorted=True):

    probd_curr=inferred_probs[:,curr_class]
    class_curr=(Y_test==curr_class)
    if(to_be_sorted):
        # Concatenate probd_curr and class_curr
        combined = np.column_stack((probd_curr, class_curr))

        # Sort by probd_curr
        combined_sorted = combined[combined[:, 0].argsort()[::-1]]

        # Divide back into probd_curr and class_curr
        probd_curr = combined_sorted[:, 0]
        class_curr = combined_sorted[:, 1]

    # Set the colors for the columns
    colors = ['red' if c == 0 else 'green' for c in class_curr]
    # Plot the column plot
    plt.bar(range(len(probd_curr)), probd_curr, color=colors)

    # Add a vertical line at value 0.5
    plt.axhline(y=0.5, color='blue', linestyle='--')



    # Add labels and title
    plt.xlabel('Index')
    plt.ylabel('Probability')
    plt.title(f'class {curr_class}')

    # Show the plot
    plt.show()

def display_feature_importance(ngb,X_train):

    shap.initjs()

    ## SHAP plot for loc trees
    explainer = shap.TreeExplainer(ngb, model_output=0) # use model_output = 1 for scale trees
    shap_values = explainer.shap_values(X_train)
    shap.summary_plot(shap_values, X_train, feature_names=X_train.columns.to_numpy())



def get_tree_hyper_params(trial):
    criterion= "friedman_mse"#trial.suggest_categorical("criterion", ["squared_error", "friedman_mse", "absolute_error", "poisson"])
    splitter="random"#trial.suggest_categorical("splitter", ["best","random"])
    max_features=None#trial.suggest_categorical("max_features", ["sqrt","log2",None])
    # max_leaf_nodesint=trial.suggest_categorical("max_leaf_nodesint", [])
    max_depth=2#trial.suggest_int("max_depth", 1,10)
    min_samples_leaf=1#trial.suggest_int("min_samples_leaf", 1,3)
    min_impurity_decrease= 0.2307277162959608#trial.suggest_float("min_impurity_decrease", 0.0,0.3)

    return sklearn.tree.DecisionTreeRegressor(criterion=criterion,splitter=splitter,max_depth=max_depth,max_features=max_features,min_samples_leaf=min_samples_leaf
                                              ,min_impurity_decrease=min_impurity_decrease)





def clasify( main_df_val,main_df_train,y_cols,chosen_y_col,num_classes,K,to_display,Base,n_estimators,learning_rate,minibatch_frac):

    main_df_val[chosen_y_col]=main_df_val[chosen_y_col].to_numpy().astype(int)
    main_df_train[chosen_y_col]=main_df_train[chosen_y_col].to_numpy().astype(int)
    
    # main_df_val = main_df_val[main_df_val[chosen_y_col] > -1]
    # main_df_train = main_df_train[main_df_train[chosen_y_col] > -1]

    # print(f"mmmm val {len(main_df_val)} train {len(main_df_train)}")

    X_train = main_df_train.drop(columns=y_cols )
    X_test = main_df_val.drop(columns=y_cols)

    X_train = X_train.iloc[:, 1:]
    X_test = X_train.iloc[:, 1:]

    Y_train = main_df_train[chosen_y_col]
    Y_test = main_df_val[chosen_y_col]


    # select top K features using mRMR
    selected_features = mrmr_classif(X=X_train, y=Y_train, K=3,n_jobs=1)
    # selected_features = mrmr_classif(X=None, y=Y_train, K=K)

    print(f"selected_features {selected_features}")
    # selected_features =['original_glcm_JointEntropy_adc', 'wavelet-HLH_firstorder_RobustMeanAbsoluteDeviation_adc', 'wavelet-LLL_firstorder_Kurtosis_adc', 'original_shape_Sphericity_adc', 'wavelet-LHL_firstorder_RootMeanSquared_hbv', 'original_glcm_SumEntropy_adc', 'log-sigma-3-0-mm-3D_glszm_SmallAreaEmphasis_adc']
    
    X_train=main_df_train[selected_features]
    X_test=main_df_val[selected_features]



    # # print(f"yyyyyyyyy {Y_train.to_numpy().astype(int)}")
    ngb_cat = NGBClassifier(Dist=k_categorical(num_classes), verbose=True
                            ,Base=Base
                            ,n_estimators=n_estimators
                            ,learning_rate=learning_rate
                            ,minibatch_frac=minibatch_frac) 
    def classsssa():
        try:
            _ = ngb_cat.fit(X_train, Y_train.to_numpy().astype(int))
        except:
            try:
                _ = ngb_cat.fit(X_train, Y_train.to_numpy().astype(int))
            except:
                try:
                    _ = ngb_cat.fit(X_train, Y_train.to_numpy().astype(int))    
                except:
                    _ = ngb_cat.fit(X_train, Y_train.to_numpy().astype(int)) 
    def classsssb():                  

        try:
            classsssa()
        except:
            classsssa() 
    def classsss():                  

        try:
            classsssb()
        except:
            classsssb() 
            
    try:
        classsss()
    except:
        classsss()    
    #     print(f"error")
    #     return 0.0
    
    file_path = Path('/workspaces/pilot_lymphoma/data/ngbtest.p')

    with file_path.open("wb") as f:
        pickle.dump(ngb_cat, f)

    # with file_path.open("rb") as f:
    #     ngb_cat = pickle.load(f)

    if(to_display):
        #display feature importance
        display_feature_importance(ngb_cat,X_train)


    inferred=ngb_cat.predict(X_test)
    # print(f"iii {inferred}")
    # print(f"iii2 {Y_test.to_numpy()}")

    acc=accuracy_score(Y_test.to_numpy(), inferred)
    inferred_probs = ngb_cat.predict_proba(X_test)

    # if(to_display):
    # for curr_class in range(num_classes):
    #     display_probs(curr_class, inferred_probs, Y_test)

    # print(f"probs {inferred_probs}")
    print(f"""AAA Accuracy: {acc}""")
    # if(num_classes==2):
    #     a=(inferred_probs[:,1]>0.7).astype(bool)
    #     b=Y_test.to_numpy()
    #     high_confidence=np.sum(np.logical_and(a,b).flatten())/np.sum(b.flatten())
    #     print(f"high_confidence {high_confidence}")








    return acc,inferred_probs, Y_test
#K is number of features we want to select
# K=20

# def classify_full(trial):
def classify_full():
    
    # K=20
    K=3
    # X, y = make_classification(n_samples = 1000, n_features = 50, n_informative = 10, n_redundant = 40)
    res_path=""
    radiomics_full_data_path="/workspaces/pilot_lymphoma/data/extracted_features_pet_trimmedB.csv"
    radiomics_full_data=pd.read_csv(radiomics_full_data_path)
    radiomics_full_data = radiomics_full_data.loc[:, ~radiomics_full_data.columns.str.contains('Unnamed', case=False)]
    radiomics_full_data = radiomics_full_data[radiomics_full_data['lesion_num'] == 1000]
    radiomics_full_data["pat_id"]=radiomics_full_data["pat_id"].astype(int)
    radiomics_full_data.columns

    full_data_table_path="/workspaces/pilot_lymphoma/data/full_table_data_for_delta.csv"
    full_data_table= pd.read_csv(full_data_table_path)
    full_data_table["pat_id"]=full_data_table["Unnamed: 0"].astype(int)
    full_data_table["outcome"]=full_data_table["Unnamed: 12"]
    # np.unique(full_data_table["outcome"].to_numpy)
    # Get first 20 percent of rows
    rows = list(full_data_table.iterrows())
    delta_r=list(map( lambda el: get_delta_radiomics(el, radiomics_full_data),rows))

    delta_r= [el for el in delta_r if el != " "]
    delta_r=pd.DataFrame(delta_r)

    # print(f"main_df_train {main_df_train['outcome']}")

    
    



    y_cols=["outcome"]#,"vol_in_mm3"
    # clinical_cols=["dre","psa","age"]
    # clinical_cols=["psa","age","dre"]
    # chosen_y_col="is_cancer"
    # chosen_y_col="isup"
    # chosen_y_col="isup_simple"
    chosen_y_col="outcome"
    # num_classes=2
    num_classes=2

    n_estimators=20#trial.suggest_int("n_estimators", 100,2000)   
    learning_rate=0.006639867572400997#trial.suggest_float("learning_rate", 0.00001,0.1)   
    minibatch_frac = 0.7561751607203051#trial.suggest_float("minibatch_frac", 0.7,1.0) 

   
    # clasify( main_df_val,main_df_train,y_cols,clinical_cols,chosen_y_col,num_classes,K)
    # clasify( main_df_val,main_df_train,y_cols,clinical_cols,chosen_y_col,num_classes,K)
    Base=get_tree_hyper_params([])    
    
    # Perform 5-fold cross validation
    kf = KFold(n_splits=5, shuffle=True)

    acc=[]
    inferred_probs=[]
    Y_test=[]
    for train_index, val_index in kf.split(delta_r):
        main_df_train = delta_r.iloc[train_index]
        main_df_val = delta_r.iloc[val_index]


        # main_df_val = delta_r.head(int(len(delta_r) * 0.2))
        # main_df_train = delta_r.tail(int(len(delta_r) * 0.8))
        
        reACC,inferred_probS_CURR,Y_test_curr=clasify( main_df_val,main_df_train,y_cols,chosen_y_col,num_classes,K,False,Base,n_estimators,learning_rate,minibatch_frac)
        
        
        inferred_probs.append(inferred_probS_CURR)
        Y_test.append(Y_test_curr)
        
        acc.append(acc)
        
        # print(res[0])
        
    print(inferred_probs)
    inferred_probs = np.concatenate(inferred_probs,axis=0)
    Y_test = np.concatenate(Y_test,axis=0)
    print(f"mmmmmmmmean acc {np.mean(acc)}")

    print(f"yyyyyyyyy inferred_probs {inferred_probs.shape} Y_test {Y_test.shape}")
    for curr_class in range(num_classes):
        display_probs(curr_class, inferred_probs, Y_test)
    
    return np.mean(acc)


    # in case of clasyfing isup we need to take a maximum of the isup values for each lesion

classify_full()

100%|██████████| 3/3 [00:00<00:00,  4.58it/s]


selected_features ['wavelet-HHH_glcm_Imc2_pet', 'log-sigma-4-0-mm-3D_firstorder_90Percentile_pet', 'wavelet-LLH_glszm_LargeAreaHighGrayLevelEmphasis_ct']
[iter 0] loss=0.7205 val_loss=0.0000 scale=8.0000 norm=17.4545
[iter 0] loss=0.5945 val_loss=0.0000 scale=4.0000 norm=7.6364
[iter 0] loss=0.5945 val_loss=0.0000 scale=512.0000 norm=977.4545
[iter 0] loss=0.6575 val_loss=0.0000 scale=512.0000 norm=1047.2727
[iter 0] loss=0.6575 val_loss=0.0000 scale=512.0000 norm=1047.2727
AAA Accuracy: 1.0


100%|██████████| 3/3 [00:00<00:00,  4.66it/s]


selected_features ['wavelet-HHH_glcm_Imc2_pet', 'wavelet-LLL_glszm_LowGrayLevelZoneEmphasis_pet', 'wavelet-HHH_firstorder_Mean_pet']
[iter 0] loss=0.5860 val_loss=0.0000 scale=8.0000 norm=16.1157
[iter 0] loss=0.4941 val_loss=0.0000 scale=512.0000 norm=920.3306
AAA Accuracy: 0.75


100%|██████████| 3/3 [00:00<00:00,  4.41it/s]


selected_features ['wavelet-HHH_glcm_Imc1_pet', 'log-sigma-3-0-mm-3D_firstorder_Kurtosis_pet', 'wavelet-HHL_glszm_GrayLevelNonUniformityNormalized_ct']
[iter 0] loss=0.5860 val_loss=0.0000 scale=2.0000 norm=4.0289
[iter 0] loss=0.5860 val_loss=0.0000 scale=4.0000 norm=8.0579
[iter 0] loss=0.5860 val_loss=0.0000 scale=2.0000 norm=4.0289
[iter 0] loss=0.5860 val_loss=0.0000 scale=32.0000 norm=64.4628
[iter 0] loss=0.6780 val_loss=0.0000 scale=2.0000 norm=4.4628
[iter 0] loss=0.5860 val_loss=0.0000 scale=2.0000 norm=4.0289
[iter 0] loss=0.4941 val_loss=0.0000 scale=512.0000 norm=920.3306
[iter 0] loss=0.5860 val_loss=0.0000 scale=2.0000 norm=4.0289
[iter 0] loss=0.5860 val_loss=0.0000 scale=512.0000 norm=1031.4050
[iter 0] loss=0.5860 val_loss=0.0000 scale=16.0000 norm=32.2314
[iter 0] loss=0.5860 val_loss=0.0000 scale=512.0000 norm=1031.4050
[iter 0] loss=0.5860 val_loss=0.0000 scale=2.0000 norm=4.0289
[iter 0] loss=0.5860 val_loss=0.0000 scale=2.0000 norm=4.0289
AAA Accuracy: 1.0


100%|██████████| 3/3 [00:00<00:00,  4.39it/s]


selected_features ['wavelet-HHH_glcm_Imc1_pet', 'wavelet-HHL_glszm_GrayLevelVariance_ct', 'log-sigma-5-0-mm-3D_glszm_LargeAreaLowGrayLevelEmphasis_ct']
[iter 0] loss=0.6780 val_loss=0.0000 scale=2.0000 norm=4.4628
[iter 0] loss=0.5860 val_loss=0.0000 scale=8.0000 norm=16.1157
[iter 0] loss=0.5860 val_loss=0.0000 scale=4.0000 norm=8.0579
[iter 0] loss=0.5860 val_loss=0.0000 scale=8.0000 norm=16.1157
[iter 0] loss=0.5860 val_loss=0.0000 scale=512.0000 norm=1031.4050
[iter 0] loss=0.4941 val_loss=0.0000 scale=8.0000 norm=14.3802
AAA Accuracy: 1.0


100%|██████████| 3/3 [00:00<00:00,  3.91it/s]